# Agenda

0. Q&A
1. `**kwargs` and other parameter types
2. Scoping (LEGB)
3. Enclosing functions
4. Dispatch tables
5. Comprehensions
6. Sorting and `lambda` (and passing functions as arguments to other functions)

In [1]:
d = {}
d['a'] = 10
hash('a') % 8

0

In [2]:
hash('b') % 8

2

In [3]:
hash('c') % 8

6

In [4]:
hash('d') % 8

3

In [5]:
for one_letter in 'abcdefghij':
    print(f'{one_letter}: {hash(one_letter) % 8}')

a: 0
b: 2
c: 6
d: 3
e: 0
f: 5
g: 6
h: 0
i: 6
j: 4


In [6]:
d['e'] = 500